## Storm Mode/Precipitation Type Classification Output for 3D Reflectivity from derived dBZ of WRF Simulations. 

**Based on the Convective/Stratiform separation on the 12 $\sigma$ level of reflectivity.**

**Calculate the max. composite reflectivity instead of REFLC_10CM from CONUS1 runs.**

**Output the Storm Mode Classification information (1: DCC; 2:WCC; 3: DWCC; 4: BSR) to CONUS dBZ data.**

**For [High Resolution WRF Simulations of the Current and Future Climate of North America](https://rda.ucar.edu/datasets/ds612.0/).**

**Hungjui Yu 20211027**

In [1]:
# import sys
# from shutil import copyfile
import time
import datetime as dt
# import pytz
from netCDF4 import Dataset # MFDataset
import numpy as np
from scipy.ndimage import label, generate_binary_structure
import xarray as xr
import wrf
from wrf import (getvar, interplevel, destagger)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
import matplotlib as mpl
import matplotlib.pyplot as plt


**Set input files paths and names:**

In [2]:
def set_input_names(file_date):

    file_path_1_conus = '/gpfs/fs1/collections/rda/data/ds612.0'
    file_path_1_dbz = '/glade/scratch/hungjui/DATA_WRF_CONUS_1_dBZ_v1.0'
    file_path_2 = '/' + wrf_sim_type # '/CTRL3D'
    file_path_3 = '/{}'.format(file_date.strftime('%Y'))

    file_names = dict( dbz = file_path_1_dbz
                           + file_path_2 
                           + '/20130913'# file_path_3 
                           + '/wrf3d_d01_' + wrf_sim_type[0:-2] + '_dbz_{}.nc'.format(file_date.strftime('%Y%m%d'))
                       , Z = file_path_1_conus
                           + file_path_2 
                           + file_path_3 
                           + '/wrf3d_d01_' + wrf_sim_type[0:-2] + '_Z_{}.nc'.format(file_date.strftime('%Y%m%d'))
                     )
    
    return file_names

**Function: Reflectivity Geo-Height Interpolation (linearly):**

In [1]:
def dbz_geoh_interp(refl, geoH, interp_lev_km):
    
    # # Use linear Z for interpolation:
    refl_linear = 10**(refl/10.)
    
    ## Interpolation:
    ## !!! convert interpolation level to the same as geo-H (meter) !!!
    refl_linear_lev = interplevel(refl_linear, geoH, interp_lev_km*1000)
    
    ## Convert back to dBz after interpolation:
    refl_lev = 10.0 * np.log10(refl_linear_lev)
    
    return refl_lev

**Function: Set Storm Mode Classification Thresholds:**

In [3]:
def set_classification_thresholds(threshold_type):
    
    ## Make sure the thresholds are either Mocderate or Strong:
    assert threshold_type in ['moderate', 'strong']
    
    if ( threshold_type == 'moderate' ):
        dbz_threshold = 30 # dBZ
        height_threshold = 8 # km
        WCC_threshold = 800 # km^2
        BSR_threshold = 40000 # km^2
    else:
        dbz_threshold = 40 # dBZ
        height_threshold = 10 # km
        WCC_threshold = 1000 # km^2
        BSR_threshold = 50000 # km^2
        
    return dbz_threshold, height_threshold, WCC_threshold


**Function: Storm Mode Classification:**

**Function: Merge masks to Storm Modes:**

In [ ]:
def merge_to_Storm_Mode( DCC_mask
                       , WCC_mask
                       , DWCC_mask
                       , BSR_mask
                       ):
    
    Storm_Mode = np.where( (DCC_mask==1), 1, 0 )
    Storm_Mode = np.where( (WCC_mask==1), 2, Storm_Mode )
    Storm_Mode = np.where( (DWCC_mask==1), 3, Storm_Mode )
    Storm_Mode = np.where( (BSR_mask==1), 4, Storm_Mode )

    # Storm_Mode[np.where(Storm_Mode == 0)] = np.nan
    
    return Storm_Mode

### Main Function:

In [6]:
def main_function(file_date_time):
    
    ## Set file datetime:
    # file_date_time = dt.datetime(2013, 9, 13, 0, 0, 0, tzinfo=pytz.utc)
    print('\nProcessing: {}'.format(file_date_time.strftime('%Y%m%d')), end=': ')
    
    ## Set input files paths and names:
    file_name_dict = set_input_names(file_date_time)

    ## Get the 3-hourly time list:
    nc_wrf_dbz = Dataset(file_name_dict['dbz'], mode='r')
    wrf_3hour_list = wrf.extract_times(nc_wrf_dbz, timeidx=wrf.ALL_TIMES, meta=False, do_xtime=False)
    nc_wrf_dbz.close()
    
    ## Open dBZ data array and append calculated data:
    ds_wrf_dbz = xr.open_dataset(file_name_dict['dbz'])
    
    ##

        
    ## Add CS mask to dBZ dataset:
    ds_wrf_dbz['CS_mask'] = (['Time', 'south_north', 'west_east'], CS_mask)
    
    ds_wrf_dbz.close()
    
    return ds_wrf_dbz


### Main Program:

In [7]:
start = time.time()


## WRF Model Simulation Category:
wrf_sim_type = 'CTRL3D'
# wrf_sim_type = 'PGW3D'

## Loop through a period:
target_date_range = pd.date_range(start='2013-9-12', end='2013-9-12', tz=pytz.utc)

for dayi in target_date_range:
        
    ## Derive Convective/Stratiform mask into dBZ dataset:
    ds_wrf_dbz = main_function(dayi)
    
    ## Add attributes to dBZ and Convective/Stratiform mask:
    ds_wrf_dbz.dBZ.attrs['long_name'] = 'Reflectivity (dBZ)'
    ds_wrf_dbz.dBZ.attrs['description'] = 'Derived radar reflectivity using wrf-python package (wrf.dbz)'
    
    ds_wrf_dbz.CS_mask.attrs['long_name'] = 'Convective/Stratiform Mask'
    ds_wrf_dbz.CS_mask.attrs['description'] = 'Derived mask for convective (1-5) and stratiform (0) echos from sigma level: 12'
    
    # ds_wrf_dbz.CS_mask_13.attrs['long_name'] = 'Convective/Stratiform Mask'
    # ds_wrf_dbz.CS_mask_13.attrs['description'] = 'Derived mask for convective (1-5) and stratiform (0) echos from sigma level: 13'
        
    
    ## Set output file path and name (to the original dBZ dataset):
    file_name_dict = set_input_names(dayi)
    file_path_name = file_name_dict['dbz']
    
    ds_wrf_dbz.to_netcdf(file_path_name, 'a')
    ds_wrf_dbz.close()

    
end = time.time()

print("RUNTIME：%f SEC" % (end - start))
print("RUNTIME：%f MIN" % ((end - start)/60))
print("RUNTIME：%f HOUR" % ((end - start)/3600))
    


Processing: 20130912: 0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  RUNTIME：72.465595 SEC
RUNTIME：1.207760 MIN
RUNTIME：0.020129 HOUR


In [8]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


<font color='teal'>**Supplement Codes:**</font>

In [9]:
# from netCDF4 import (Dataset, MFDataset)
# nc_wrf_dbz = Dataset( '/glade/scratch/hungjui/DATA_WRF_CONUS_1_dBZ_v1.0/CTRL3D/20130913/wrf3d_d01_CTRL_dbz_20130913.nc'
#                     , mode='a')
# # nc_wrf_dbz.createDimension('test_dim', 1)
# print(nc_wrf_dbz)
# nc_wrf_dbz.close()